In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [3]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
import qiskit.providers.aer.noise as noise
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts
from vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits, organize_tomography_data_from_list

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.3.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

qubits = 5

simulation_backend = "qasm_simulator"

seed = 0
print_circuits = True

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [7]:
device = provider.get_backend('ibm_cairo')
device

<IBMQBackend('ibm_cairo') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [8]:
device.configuration().to_dict()['max_shots']

100000

In [9]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [10]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [11]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_front.qasm")

In [12]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [13]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [14]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68590/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [15]:
orign_circuit.draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├──────────────────────────────────────────────■────────»
     └───────────────┘                                                       »
q_4: ────────────────────────────────────────────────────────────────────────»
                                                                             »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_4: ─────────────────
«

In [16]:
#checking circuit

In [17]:
def apply_checking_circuit(qc, ctrl_bits, ancilla_bits, side = None):
    if len(ctrl_bits) != len(ancilla_bits):
        print("Size mismatch")
        return None
    if side == 'front':
        for i in ancilla_bits:
            qc.h(i)
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
    elif side == 'end':
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
        for i in ancilla_bits:
            qc.h(i)
    else:
        print("Side undefined")

In [18]:
circuit = front_layer.copy()

In [19]:
apply_checking_circuit(circuit, [2], [4], side = 'front')
circuit += mid_layer
apply_checking_circuit(circuit, [2], [4], side = 'end')
circuit += end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68590/2062351668.py:2: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += mid_layer
/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68590/2062351668.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += end_layer


In [20]:
circuit.draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────■───────────────────■─────────────────■────────»
     ├───────────────┤        │                                     │        »
q_3: ┤ Ry(-0.019851) ├────────┼─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                         
«q_0: ────────────────────
«                         
«q_1: ────────────────────
«        ┌───────────────┐
«q_2: ─■─┤ Ry(0.0036205) ├
«      │ └┬──────────────┤
«q_3: ─┼──┤ Ry(0.019873) ├
«      │  └────┬───┬─────┘
«q_4: ─■───────┤ H ├──────
«              └───┘

In [21]:
pauli_list = read_from_file('LiH_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [22]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4,num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
                     
q_3: ────────────────
                     
q_4: ────────────────
                     
ZZYY
                     
q_0: ────────────────
                     
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZX
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
      ┌────────────┐ 
q_3: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_4: ────────────────
                     
ZXYY
       

In [23]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68590/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [24]:
circuit_list[-2].draw()

┌───┐      ┌───────────────┐   ┌────────────────┐  ┌────────────┐ »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├──┤ U(π/2,0,π) ├─»
           ├───┤      ├───────────────┤ │ └────────────────┘┌─┴────────────┴┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────■───────────────────■─────────────────■────────»
     ├───────────────┤        │                                     │        »
q_3: ┤ Ry(-0.019851) ├────────┼─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                                  
«q_0: ─────────────────────────────────────────────
«     ┌────────────┐                               
«q_1: ┤ U(π/2,0,π) ├───────────────────────────────
«     └────────────┘┌───────────────┐┌────────────┐
«q_2: ──────■───────┤ Ry(0.0036205) ├┤ U(π/2,0,π) ├
«           │       └┬──────────────┤├────────────┤
«q_3: ──────┼────────┤ Ry(0.019873) ├┤ U(π/2,0,π) ├
«           │        └────┬───┬─────┘└────────────┘
«q_4: ──────■─────────────┤ H ├────────────────────
«                         └───┘

In [25]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [26]:
cuts = [(circuit.qubits[2], 2),(circuit.qubits[2], 4)]

In [27]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

           ┌───┐      ┌───────────────┐   ┌────────────────┐ ┌──────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┤ U(π/2,0,π/2) ├»
           ├───┤      ├───────────────┤ │ └────────────────┘┌┴──────────────┤»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────■───────────────────■─────────────────■────────»
     ├───────────────┤        │                                     │        »
q_3: ┤ Ry(-0.019851) ├────────┼─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                                      
«q_0: ─────────────────────────────────────────────────
«     ┌────────────

           ┌───┐      ┌───────────────┐   ┌────────────────┐ ┌──────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┤ U(π/2,0,π/2) ├»
           ├───┤      ├───────────────┤ │ └────────────────┘┌┴──────────────┤»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────■───────────────────■─────────────────■────────»
     ├───────────────┤        │                                     │        »
q_3: ┤ Ry(-0.019851) ├────────┼─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                                    
«q_0: ───────────────────────────────────────────────
«     ┌──────────────┐ 

           ┌───┐      ┌───────────────┐   ┌────────────────┐  ┌────────────┐ »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├──┤ U(π/2,0,π) ├─»
           ├───┤      ├───────────────┤ │ └────────────────┘┌─┴────────────┴┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────■───────────────────■─────────────────■────────»
     ├───────────────┤        │                                     │        »
q_3: ┤ Ry(-0.019851) ├────────┼─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                                  
«q_0: ─────────────────────────────────────────────
«     ┌────────────┐       

In [28]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 0)),), Qubit(QuantumRegister(5, 'q'), 1): ((0, Qubit(QuantumRegister(4, 'q'), 3)),), Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(3, 'q'), 1)), (0, Qubit(QuantumRegister(4, 'q'), 2)), (1, Qubit(QuantumRegister(3, 'q'), 2))), Qubit(QuantumRegister(5, 'q'), 3): ((0, Qubit(QuantumRegister(4, 'q'), 1)),), Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 0)),)}


In [29]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [30]:
total_variants

24

In [31]:
shots = device.configuration().to_dict()['max_shots'] * total_variants

In [32]:
fragments_list[0][0].draw()

┌───┐      ┌───────────────┐   ┌────────────────┐ ┌──────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┤ U(π/2,0,π/2) ├»
     ┌─────┴───┴─────┐└───────────────┘ │ └────────────────┘ └──────────────┘»
q_1: ┤ Ry(-0.019851) ├──────────────────┼───────────────────────────■────────»
     └───────────────┘                  │                           │        »
q_2: ───────────────────────────────────┼─────────■─────────────────■────────»
           ┌───┐      ┌───────────────┐ │         │         ┌───────────────┐»
q_3: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
           └───┘      └───────────────┘                     └───────────────┘»
«                                     
«q_0: ────────────────────────────────
«     ┌──────────────┐┌──────────────┐
«q_1: ┤ Ry(0.019873) ├┤ U(π/2,0,π/2) ├
«     └──────────────┘└──────────────┘
«q_2: ────────────────────────────────
«     ┌──────────────┐                
«q_3: ┤ U(π/2,0,π/2) ├────────────────
«     └──────────────┘

In [33]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
           ┌───┐      ┌───────────────┐   ┌────────────────┐ ┌──────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┤ U(π/2,0,π/2) ├»
     ┌─────┴───┴─────┐└───────────────┘ │ └────────────────┘ └──────────────┘»
q_1: ┤ Ry(-0.019851) ├──────────────────┼───────────────────────────■────────»
     └───────────────┘                  │                           │        »
q_2: ───────────────────────────────────┼─────────■─────────────────■────────»
           ┌───┐      ┌───────────────┐ │         │         ┌───────────────┐»
q_3: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
           └───┘      └───────────────┘                     └───────────────┘»
«                                     
«q_0: ────────────────────────────────
«     ┌──────────────┐┌──────────────┐
«q_1: ┤ Ry(0.019873) ├┤ U(π/2,0,π/2) ├
«     └──────────────┘└──────────────┘
«q_2: ────────────────────────────────
«     ┌──────────────┐                
«q_3

In [34]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [35]:
wire_path_map

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 0)),),
 Qubit(QuantumRegister(5, 'q'), 1): ((0, Qubit(QuantumRegister(4, 'q'), 3)),),
 Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(3, 'q'), 1)),
  (0, Qubit(QuantumRegister(4, 'q'), 2)),
  (1, Qubit(QuantumRegister(3, 'q'), 2))),
 Qubit(QuantumRegister(5, 'q'), 3): ((0, Qubit(QuantumRegister(4, 'q'), 1)),),
 Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 0)),)}

In [36]:
num_fragments = len(fragments)
num_fragments

2

In [37]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

25

In [38]:
fragments[0].draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
     ┌─────┴───┴─────┐└───────────────┘ │ └────────────────┘                 »
q_1: ┤ Ry(-0.019851) ├──────────────────┼───────────────────────────■────────»
     └───────────────┘                  │                           │        »
q_2: ───────────────────────────────────┼─────────■─────────────────■────────»
           ┌───┐      ┌───────────────┐ │         │         ┌───────────────┐»
q_3: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
           └───┘      └───────────────┘                     └───────────────┘»
«                     
«q_0: ────────────────
«     ┌──────────────┐
«q_1: ┤ Ry(0.019873) ├
«     └──────────────┘
«q_2: ────────────────
«                     
«q_3: ────────────────
«

In [39]:
fragments[1].draw()

┌───┐                  ┌───┐      
q_0: ──────┤ H ├───────■──■───────┤ H ├──────
     ┌─────┴───┴─────┐ │  │       └───┘      
q_1: ┤ Ry(0.0038243) ├─■──┼──────────────────
     └───────────────┘    │ ┌───────────────┐
q_2: ─────────────────────■─┤ Ry(0.0036205) ├
                            └───────────────┘

In [40]:
hardware_index = 0

In [42]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend, extra_qc = None)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [43]:
#contains 25 basis
len(tomography_circuits)

25

In [44]:
#contains three fragments
len(tomography_circuits[0])

2

In [45]:
len(tomography_circuits[0][1])

1

In [46]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [47]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [48]:
multi_exp_circuits[0][-200].draw()

░       ┌───┐      ┌───────────────┐   »
  q_0: ────────────────────░───────┤ X ├──────┤ Ry(-0.003534) ├─■─»
                           ░ ┌─────┴───┴─────┐└───────────────┘ │ »
  q_1: ────────────────────░─┤ Ry(-0.019851) ├──────────────────┼─»
       ┌─────────────────┐ ░ └───────────────┘                  │ »
  q_2: ┤ U3(-1.9106,π,0) ├─░────────────────────────────────────┼─»
       └─────────────────┘ ░       ┌───┐      ┌───────────────┐ │ »
  q_3: ────────────────────░───────┤ X ├──────┤ Ry(-0.015869) ├─■─»
                           ░       └───┘      └───────────────┘   »
c66_0: ═══════════════════════════════════════════════════════════»
                                                                  »
c66_1: ═══════════════════════════════════════════════════════════»
                                                                  »
c66_2: ═══════════════════════════════════════════════════════════»
                                                                  »
c66_3: ═══════════════════════════════════════════════════════════»
                                                                  »
«       ┌────────────────┐ ┌──────────────┐                               ░ ┌─┐»
«  q_0: ┤ Ry(-0.0036834) ├─┤ U(π/2,0,π/2) ├───────────────────────────────░─┤M├»
«       └────────────────┘ └──────────────┘┌──────────────┐┌────────────┐ ░ └╥┘»
«  q_1: ──────────────────────────■────────┤ Ry(0.019873) ├┤ U(π/2,0,π) ├─░──╫─»
«                                 │        └──────────────┘└────────────┘ ░  ║ »
«  q_2: ────────■─────────────────■───────────────────────────────────────░──╫─»
«               │         ┌───────────────┐┌──────────────┐               ░  ║ »
«  q_3: ────────■─────────┤ Ry(-0.015751) ├┤ U(π/2,0,π/2) ├───────────────░──╫─»
«                         └───────────────┘└──────────────┘               ░  ║ »
«c66_0: ═════════════════════════════════════════════════════════════════════╩═»
«                                                                              »
«c66_1: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«c66_2: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«c66_3: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«                  
«  q_0: ───────────
«       ┌─┐        
«  q_1: ┤M├────────
«       └╥┘┌───┐┌─┐
«  q_2: ─╫─┤ H ├┤M├
«        ║ └┬─┬┘└╥┘
«  q_3: ─╫──┤M├──╫─
«        ║  └╥┘  ║ 
«c66_0: ═╬═══╬═══╬═
«        ║   ║   ║ 
«c66_1: ═╩═══╬═══╬═
«            ║   ║ 
«c66_2: ═════╬═══╩═
«            ║     
«c66_3: ═════╩═════
«

In [49]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [23,26,25,24], 
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

job_id:  6377131018d7482322f7531e
Job Status: job is queued (126)   

IBMQJobApiError: '"HTTPSConnectionPool(host=\'api-qcon.quantum-computing.ibm.com\', port=443): Max retries exceeded with url: /api/Network/ibm-q-ornl/Groups/ornl/Projects/chm185/Jobs/6377131018d7482322f7531e/status/v/1 (Caused by NewConnectionError(\'<urllib3.connection.HTTPSConnection object at 0x7f8709dfb280>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known\'))"'

In [ ]:
frag_datas[0][0].results[0].header.name

In [ ]:
num_tomo_circuits

In [ ]:
test = [[], []]

In [ ]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [ ]:
frag_datas[0][0].get_counts()

In [ ]:
frag_datas[0][0].get_counts()

In [ ]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

In [ ]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

In [ ]:
len(organized_data)

In [ ]:
organized_data[2]

In [ ]:
organized_data[-1][-1]

In [ ]:
recombined_lists = []
filtered_recombined_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))

In [ ]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [ ]:
direct_recombined_dist

In [ ]:
likely_recombined_dist

In [ ]:
filtered_recombined_lists

In [ ]:
from utils.utils import filter_results

In [ ]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [ ]:
filter_direct_recombined

In [ ]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [ ]:
filter_likely_recombined

In [ ]:
def norm_dict(dictionary):
    total = total_counts(dictionary)
    norm_dist = {}
    for i in dictionary.keys():
        norm_dist[i] = dictionary[i]/total
    return norm_dist

In [ ]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [ ]:
likely_recombined_dist['00011']

In [ ]:
#normalized distribution after mitigation: 0.9189375->0.9286796265616319
norm_filter_dist['0011']

In [ ]:
norm_filter_dist

In [ ]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(filtered_recombined_lists[i], shots = 1, Pauli = Pauli_tuple[0])  

In [ ]:
final_expect_val